In [2]:
# dataanalisys stack

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import stat
import scipy
import re

# SO
import os


# setando nível de log warning
import warnings
warnings.filterwarnings('ignore')



## Carregando consultas

In [4]:
# carregando dados de consultas
consultas_path = "../../input_data/7560/7560/"

print(len(os.listdir(consultas_path)))

28


In [5]:
consulta = pd.DataFrame()
for file in os.listdir(consultas_path):
    if file.endswith('.csv'):
        try:
            consulta = pd.concat([consulta, pd.read_csv(consultas_path + file, sep=';')]) 
        except:
            print(file)

In [6]:
refat_status = {'Não foi possível consultar o saldo no momento! - Instituição Fiduciária não possui autorização do Trabalhador para Operação Fiduciária.': 'falta autorizacao',
'Trabalhador não possui adesão ao saque aniversário vigente na data corrente.': 'sem adesão ao SA',
'Existe uma Operação Fiduciária em andamento. Tente mais tarde.': 'Já existe operação',
'Não foi possível consultar o saldo no momento! - Trabalhador informado não possui contas de FGTS.': 'não tem conta FGTS',
'Não foi possível consultar o saldo no momento! - Mudanças cadastrais na conta do FGTS foram realizadas, que impedem a contratação. Entre em contato com o setor de FGTS da CAIXA.': 'mudança cadastrais',
'Não foi possível consultar o saldo no momento!': 'sem consulta',
'Endpoint request timed out': 'timeout',
'Não foi possível consultar o saldo no momento! - Existe uma Operação Fiduciária em andamento. Tente mais tarde.': 'Já existe operação',
'Too Many Requests': 'too many requests',
'Não foi possível consultar o saldo no momento! - Operação não permitida por pendência no processo de pagamento de saque aniversário.': 'pendência no processo de pagamento SA',
'Não foi possível consultar o saldo no momento! - Não é possível realizar a operação para o CPF informado.': 'proibida operação CPF informado',
'Não foi possível consultar o saldo no momento! - Operação não permitida antes de 13/05/2025. ': 'data limit',
'Não foi possível consultar o saldo no momento! - Operação não permitida antes de 10/05/2025. ': 'data limit',
'Request failed with status code 504':'504'}

In [7]:
consulta.message = consulta.message.map(refat_status)

In [8]:
consulta.sample(10)

,id,provider,document_number,status,provider_key,created_at,updated_at,partner_id,message
30268,65366956-39fb-4aa6-9566-f1e4f7b0b801,bms,6180087300,failed,65366956-39fb-4aa6-9566-f1e4f7b0b801,2025-09-04 04:15:19.207673,2025-09-04 04:15:19.207673,7560_7715_D092,NaN
52598,5538ee3e-d5e6-484b-a017-2826f0034ef6,bms,46664649802,completed,5538ee3e-d5e6-484b-a017-2826f0034ef6,2025-09-28 11:20:25.706724,2025-09-28 11:20:25.706724,7560.01_D107,NaN
72288,2cb07b45-e38d-4cc3-ba8b-134ec0737112,bms,966957067,failed,2cb07b45-e38d-4cc3-ba8b-134ec0737112,2025-09-27 21:39:50.789763,2025-09-27 21:39:50.789763,7560_7715_D087,NaN
114242,924a2277-70f7-4bfb-b151-000be9a3690b,bms,27637051897,failed,924a2277-70f7-4bfb-b151-000be9a3690b,2025-09-02 17:17:14.231310,2025-09-02 17:17:14.231310,7560_7589_S019,NaN
27625,aa5bb8ea-0776-4b6b-b46f-de0be36821dd,bms,2506761179,failed,aa5bb8ea-0776-4b6b-b46f-de0be36821dd,2025-09-08 05:15:43.106914,2025-09-08 05:15:43.106914,7560_7576_D009,too many requests
12058,5ccf0b70-3210-4cf2-88f7-3d9a372bed32,bms,12080577662,failed,5ccf0b70-3210-4cf2-88f7-3d9a372bed32,2025-09-27 03:25:15.345143,2025-09-27 03:25:15.345143,7560.01_D012,too many requests
4044,c01a3e1b-ddbd-4b53-bce6-bc723f771bbb,cartos,76691390059,completed,0b8795f1-9893-41ba-8720-6953e9d95d0c,2025-09-17 01:11:56.642497,2025-09-17 01:11:58.973494,7560.01_11610,NaN
53179,37e352ae-08ec-45c0-ae68-7e103a18f0c4,cartos,70725140232,failed,ce33b7fc-13db-4f1d-8839-9d13e4d75ab8,2025-09-24 10:32:32.659163,2025-09-24 10:40:07.223563,7560.01_11610_S001,NaN
96719,0b5c8481-6b91-45df-b248-55189f84eb0f,bms,2608158455,failed,0b5c8481-6b91-45df-b248-55189f84eb0f,2025-09-28 23:43:07.229250,2025-09-28 23:43:07.229250,7560.01_D081,NaN
25276,e621785d-020d-4628-b040-d66ac3191fa2,bms,9321769170,failed,e621785d-020d-4628-b040-d66ac3191fa2,2025-09-25 04:36:18.839167,2025-09-25 04:36:18.839167,7560.01_D093,too many requests


In [11]:
consulta = consulta[
    (consulta.message != 'falta autorizacao') 
    & (consulta.message != 'too many requests')
    & (consulta.message != 'timeout')]

In [12]:
consulta.message.value_counts()

message
Já existe operação    3031
sem adesão ao SA         9
Name: count, dtype: int64

In [13]:
consulta.shape

(2286609, 9)

In [14]:
consulta.sample(5)

,id,provider,document_number,status,provider_key,created_at,updated_at,partner_id,message
29513,46c2251d-4a2a-42a5-a1b5-84e4868d0f19,bms,32248051883,failed,46c2251d-4a2a-42a5-a1b5-84e4868d0f19,2025-09-15 07:00:59.024568,2025-09-15 07:00:59.024568,7560_7576_D017,NaN
84013,8f37aeac-81f6-4e68-9965-dae59f629c91,bms,4333758173,failed,8f37aeac-81f6-4e68-9965-dae59f629c91,2025-09-11 15:21:01.737386,2025-09-11 15:21:01.737386,7560.01_D069,NaN
84112,755dc742-5c0b-4c1d-b296-306925476ef2,bms,15617590820,failed,755dc742-5c0b-4c1d-b296-306925476ef2,2025-09-13 19:47:46.966056,2025-09-13 19:47:46.966056,7560.01_7560_D001,NaN
68073,aaeb7b78-1e9e-4d13-a567-1c73f11d6252,bms,11812161760,failed,aaeb7b78-1e9e-4d13-a567-1c73f11d6252,2025-09-10 16:44:20.477228,2025-09-10 16:44:20.477228,7560.01_D068,NaN
58590,1d207cfb-af84-499e-9f16-813eb212166f,bms,49826643823,failed,1d207cfb-af84-499e-9f16-813eb212166f,2025-09-14 11:11:13.148418,2025-09-14 11:11:13.148418,7560.01_D018,NaN


In [15]:
consulta.columns = ['id_consulta', 'provider_consulta', 'CPF_consulta', 'status_consulta', 'provider_key_consulta', 'created_consulta', 'update_consulta', 'partiner_consulta', 'message_consulta']

In [16]:
consulta.to_csv('../../output_data/datasets/dados_consulta.csv', index=False)

## Carregando propostas pagas

In [17]:
pagas_path = "../../input_data/modulo_operacao_bms_22-10-2025.csv"

propostas_pagas = pd.read_csv(pagas_path, header=None)

In [18]:
propostas_pagas.columns = ['id_pagas', 'date_pagas', 'undefined_pagas', 'CPF_pagas', 'provider_pagas', 'contrato_pagas', 'partiner_pagas', 'proventos_pagas', 'tabela']

In [19]:
propostas_pagas.shape

(15288, 9)

In [20]:
propostas_pagas.sample(5)

,id_pagas,date_pagas,undefined_pagas,CPF_pagas,provider_pagas,contrato_pagas,partiner_pagas,proventos_pagas,tabela
3263,0199348b-5f24-7383-bb9c-acbd600eb038,2025-09-10,1757523960,79152325253,BMS,C253465HRRCJNT,7560.01_D081,"{""tac"": {""Percentual"": 32.0}, ""spread"": {""Perc...",9af29ed0-dabd-44ec-b6ed-c70f6f03224c
5385,019953df-55b9-7271-8989-6ac22a380553,2025-09-16,1758049440,27244698879,BMS,C254766EFHHFAS,7560.01_D104,"{""tac"": {""Percentual"": 32.0}, ""spread"": {""Perc...",9af29ed0-dabd-44ec-b6ed-c70f6f03224c
5330,0199533a-2de8-7fd0-8e25-fb69dfb9b635,2025-09-16,1758045060,2528719019,BMS,C255698SJPZAWX,7560.01_D062,"{""tac"": {""Percentual"": 32.0}, ""spread"": {""Perc...",9af29ed0-dabd-44ec-b6ed-c70f6f03224c
11102,0199788a-8cd0-7eb2-8d25-4180c6cf599d,2025-09-23,1758667743,3831607079,BMS,C256663ZYWWMNM,7560.01_D106,"{""tac"": {""Percentual"": 35.0}, ""spread"": {""Perc...",9af29ed0-dabd-44ec-b6ed-c70f6f03224c
9591,01997643-c668-7f01-9caf-d418b21460c9,2025-09-23,1758626641,7797310777,BMS,C258489MUMZNGX,7560.01,"{""tac"": {""Percentual"": 44.0}, ""spread"": {""Perc...",6f5131ce-801b-4621-b65a-e73b7b34a3ff


In [21]:
## Dados de pagamento duplicados - erro no relacionamento na extracao dos dados
#propostas_pagas[propostas_pagas.document_number.duplicated()].sort_values(by='document_number')

# dropando duplicados para seguir com planejamento
propostas_pagas = propostas_pagas.drop_duplicates(subset='CPF_pagas', keep='first')

In [22]:
propostas_pagas.shape

(14692, 9)

In [23]:
propostas_pagas.to_csv('../../output_data/datasets/dados_pagas.csv', index=False)

## Merge das propostas

In [24]:
merged_data = pd.merge(right=consulta, left=propostas_pagas, how='right', right_on='CPF_consulta', left_on='CPF_pagas')

In [25]:
merged_data['CPF_consulta'] = merged_data['CPF_consulta'].astype(str).str.zfill(11)

In [26]:
merged_data.to_csv('../../output_data/datasets/merge_consulta_pagas.csv', index=False)

## Buscando comissao repassada - relatórios storm

In [27]:
cms_repassada_storm = "Y:/FINANCEIRO/Fagner/powerBI/src/download/2025/9/extra/"

storm = pd.DataFrame()
for file in os.listdir(cms_repassada_storm):
    try:
        dados_1 = pd.read_html(cms_repassada_storm + file, header=0, thousands = '.', decimal = ',')[0]
        dados_1.ValorBase.replace(",", "", regex=True, inplace=True)
        dados_1.ValorBase = dados_1.ValorBase.astype(float)
        storm = pd.concat([storm, dados_1])
    except:
        print(file)

2025-09-06.xls
2025-09-07.xls
2025-09-13.xls
2025-09-14.xls
2025-09-20.xls
2025-09-21.xls
2025-09-27.xls
2025-09-28.xls


In [29]:
storm.drop_duplicates(subset=['CPFCliente'], inplace=True, keep='first')

In [30]:
storm['Numero_Corretor'] = storm['Numero_Corretor'].astype(str)

In [31]:
storm = storm[storm['Numero_Corretor'].str.contains('7560')]

In [32]:
storm = storm[['CPFCliente', 'ADE', 'CMSRepassada']]

In [33]:
storm.CPFCliente = storm.CPFCliente.map(lambda x: re.sub(r"\D", "", x))

In [34]:
storm.columns = ['CPF_storm', 'ADE', 'CMSRepassada']

In [35]:
storm.sample(5)

,CPF_storm,ADE,CMSRepassada
17292,11355925940,373C256434YAPKPXG,29.12
5253,02172367907,C254228BACQFZ8,47.18
1221,73582751104,C252843GUBNHB8,34.13
8968,05297557933,373C258794NEENGTY,14.57
7788,08506347920,C256732WRCMPTH,18.43


In [36]:
merged_data.sample(5)

,id_pagas,date_pagas,undefined_pagas,CPF_pagas,provider_pagas,contrato_pagas,partiner_pagas,proventos_pagas,tabela,id_consulta,provider_consulta,CPF_consulta,status_consulta,provider_key_consulta,created_consulta,update_consulta,partiner_consulta,message_consulta
71951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9a3b294c-1732-4927-9d7c-85c3ad19bdf7,bms,00097051047,failed,9a3b294c-1732-4927-9d7c-85c3ad19bdf7,2025-09-01 22:52:44.464036,2025-09-01 22:52:44.464036,7560_7715_D096,NaN
766577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2a200ab8-2203-4da4-bac8-ef8c8c1175fb,bms,07647303988,failed,2a200ab8-2203-4da4-bac8-ef8c8c1175fb,2025-09-10 05:54:03.596508,2025-09-10 05:54:03.596508,7560.01_D081,NaN
338536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09c09ec2-6a4b-4019-b7e2-d9d3f66e3979,cartos,17759910754,failed,c6950db9-7249-43ef-a181-19d32e33a305,2025-09-04 21:06:31.592209,2025-09-04 21:09:05.092612,7560.01_11610_S001,NaN
1661844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ccc6474e-5e7b-4053-9bc9-88dfdea1b80c,bms,12359667637,failed,ccc6474e-5e7b-4053-9bc9-88dfdea1b80c,2025-09-19 19:47:33.024306,2025-09-19 19:47:33.024306,7560_7576_D002,NaN
2198227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23a72b2a-562b-4228-81dd-daf30f0129a8,bms,61015001360,failed,23a72b2a-562b-4228-81dd-daf30f0129a8,2025-09-27 23:17:57.228701,2025-09-27 23:17:57.228701,7560_7576_D002,NaN


In [37]:
all_Data = pd.merge(left=merged_data, right=storm, how='left', left_on='CPF_consulta', right_on='CPF_storm')

In [38]:
all_Data.to_csv('../../output_data/datasets/all_data.csv', index=False)